# MMDet-AITOD Training and Testing Notebook for Google Colab

This notebook trains and tests models using data from Google Drive.

## Setup Instructions:
1. Upload your files to Google Drive:
   - `mmdet-nwdrka` repository
   - Config files (or they can be in the repo)
   - Dataset (AI-TODv2)
2. Update the paths in CELL 1 according to your Drive structure
3. Run all cells in order


In [1]:
# ============================================================================
# CELL 1: Mount Google Drive and Configure Paths
# ============================================================================
from google.colab import drive
drive.mount('/content/drive')

# Configure your paths here
DRIVE_ROOT = '/content/drive/MyDrive'
REPO_PATH = f'{DRIVE_ROOT}/mmdet_nwdrka/mmdet-nwdrka'
COCOAPI_PATH = f'{DRIVE_ROOT}/mmdet_nwdrka/cocoapi-aitod-master/aitodpycocotools'
REQUIREMENTS_PATH = f'{DRIVE_ROOT}/mmdet_nwdrka/requirements_mmdet.txt'
DATASET_EXTRACTED_PATH = f'{DRIVE_ROOT}/mmdet_nwdrka/mmdet-nwdrka/data/aitod_super'
OUTPUT_PATH = f'{DRIVE_ROOT}/outputs'

# Python 3.8 virtual environment path
VENV_PATH = '/content/py38'
VENV_ACTIVATE = f'source {VENV_PATH}/bin/activate'
VENV_PYTHON = f'{VENV_PATH}/bin/python'

# Training configuration
CONFIG_FILE = 'nwd_rka/aitod_super_faster_r50_iou_1x.py'  # Change this to your config

print("Configuration:")
print(f"  Repository: {REPO_PATH}")
print(f"  COCOAPI: {COCOAPI_PATH}")
print(f"  Requirements: {REQUIREMENTS_PATH}")
print(f"  Dataset location: {DATASET_EXTRACTED_PATH}")
print(f"  Python 3.8 venv: {VENV_PATH}")
print(f"  Config: {CONFIG_FILE}")
print(f"  Output: {OUTPUT_PATH}")


Mounted at /content/drive
Configuration:
  Repository: /content/drive/MyDrive/mmdet_nwdrka/mmdet-nwdrka
  COCOAPI: /content/drive/MyDrive/mmdet_nwdrka/cocoapi-aitod-master/aitodpycocotools
  Requirements: /content/drive/MyDrive/mmdet_nwdrka/requirements_mmdet.txt
  Dataset location: /content/drive/MyDrive/mmdet_nwdrka/mmdet-nwdrka/data/aitod_super
  Python 3.8 venv: /content/py38
  Config: nwd_rka/aitod_super_faster_r50_iou_1x.py
  Output: /content/drive/MyDrive/outputs


In [2]:
# ============================================================================
# CELL 2: Setup Python 3.8 Environment and Install Dependencies
# ============================================================================
import os
import shutil
import sys

print(">>> 1) Python 3.8 kuruluyor...")
!apt-get update -y
!apt-get install -y python3.8 python3.8-venv python3.8-distutils

print(f"\n>>> 2) Python 3.8 venv oluşturuluyor: {VENV_PATH}")
!python3.8 -m venv {VENV_PATH}

print(f"\n>>> 3) Venv'e giriliyor ve pip/setuptools/wheel güncelleniyor...")
!{VENV_ACTIVATE} && python -m pip install --upgrade pip setuptools wheel

print("\n>>> 4) PyTorch 1.13.1 + cu117 kuruluyor...")
!{VENV_ACTIVATE} && python -m pip install \
  torch==1.13.1+cu117 \
  torchvision==0.14.1+cu117 \
  torchaudio==0.13.1 \
  -f https://download.pytorch.org/whl/torch_stable.html

print("\n>>> 5) openmim ve mmcv-full==1.7.1 kuruluyor...")
!{VENV_ACTIVATE} && python -m pip install -U openmim
!{VENV_ACTIVATE} && mim install "mmcv-full==1.7.1"

print("\n>>> 6) mmdet kuruluyor...")
!{VENV_ACTIVATE} && mim install mmdet

# Install other dependencies from requirements.txt in Drive
if os.path.exists(REQUIREMENTS_PATH):
    print(f"\n>>> 7) Found requirements.txt at: {REQUIREMENTS_PATH}")
    # Copy to local directory for installation
    LOCAL_REQUIREMENTS = '/content/requirements.txt'
    shutil.copy(REQUIREMENTS_PATH, LOCAL_REQUIREMENTS)

    # Read requirements and filter out mmcv and mmdet (already installed via mim)
    with open(LOCAL_REQUIREMENTS, 'r') as f:
        lines = f.readlines()

    # Filter out mmcv and mmdet related packages
    filtered_lines = []
    for line in lines:
        line_stripped = line.strip().lower()
        if line_stripped and not line_stripped.startswith('#'):
            # Skip mmcv and mmdet packages
            if 'mmcv' not in line_stripped and 'mmdet' not in line_stripped:
                filtered_lines.append(line)

    # Write filtered requirements
    if filtered_lines:
        FILTERED_REQUIREMENTS = '/content/requirements_filtered.txt'
        with open(FILTERED_REQUIREMENTS, 'w') as f:
            f.writelines(filtered_lines)
        print("Installing other dependencies from requirements.txt (excluding mmcv/mmdet)...")
        !{VENV_ACTIVATE} && pip install -r {FILTERED_REQUIREMENTS}
        print("✓ Other dependencies installed from requirements.txt")
    else:
        print("All dependencies in requirements.txt are mmcv/mmdet (already installed)")
else:
    print(f"\n>>> 7) Note: requirements.txt not found at {REQUIREMENTS_PATH}")
    print("Installing common dependencies...")
    !{VENV_ACTIVATE} && pip install matplotlib numpy pycocotools six terminaltables opencv-python pillow

# Verify installation
print("\n>>> 8) Installation verification...")
!{VENV_ACTIVATE} && python -c "import sys, torch, mmcv; print(f'Python: {sys.version}'); print(f'PyTorch: {torch.__version__}'); print(f'CUDA available: {torch.cuda.is_available()}'); print(f'MMCV: {mmcv.__version__}')"

# Configure notebook to use Python 3.8 venv for inline Python code
print("\n>>> 9) Configuring notebook to use Python 3.8 venv...")
venv_python = f'{VENV_PATH}/bin/python'
venv_site_packages = f'{VENV_PATH}/lib/python3.8/site-packages'

# Add venv site-packages to Python path
if venv_site_packages not in sys.path:
    sys.path.insert(0, venv_site_packages)
    print(f"✓ Added {venv_site_packages} to sys.path")

# Verify we can import from venv
try:
    import subprocess
    result = subprocess.run([venv_python, '--version'], capture_output=True, text=True)
    print(f"✓ Python 3.8 venv available: {result.stdout.strip()}")
except:
    print("Note: Could not verify venv Python version")

print("\n>>> BİTTİ.")
print(f"\nBu ortamı kullanmak için: {VENV_ACTIVATE}")
print("Notebook Python path configured to use venv packages.")


>>> 1) Python 3.8 kuruluyor...
Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,225 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,860 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,287 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,572 kB]
Hit:13 https://ppa.la

In [ ]:
# pip install torch1.13.0 cuda

#!pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu117
#!pip install -U pip setuptools wheel
#!pip install -f https://download.openmmlab.com/mmcv/dist/cu117/torch1.13/index.html "mmcv==1.7.1"


In [3]:
%%bash
set -e
echo ">> Python 3.8 ortamı aktive ediliyor..."
source /content/py38/bin/activate

echo ">> Python 3.8 geliştirme dosyaları yükleniyor..."
apt-get install -y python3.8-dev

echo ">> Mevcut aitodpycocotools uninstall ediliyor..."
python -m pip uninstall -y aitodpycocotools || true

echo ">> pip, cython, numpy, yapf ayarlanıyor..."
python -m pip install --upgrade pip setuptools
python -m pip install "cython<3.0.0" "numpy<1.24" "yapf==0.30.0"

echo ">> aitodpycocotools GitHub'dan (NO local path!) kuruluyor..."
python -m pip install -v --no-build-isolation \
  "git+https://github.com/jwwangchn/cocoapi-aitod.git#subdirectory=aitodpycocotools"

echo ">> Kurulum testi:"
python - << 'EOF'
import sys, pkg_resources
print("Python:", sys.version)
try:
    import aitodpycocotools
    print("aitodpycocotools OK:", aitodpycocotools.__file__)
except Exception as e:
    print("aitodpycocotools IMPORT FAILED:", e)

try:
    import yapf
    print("yapf version:", pkg_resources.get_distribution("yapf").version)
except Exception as e:
    print("yapf IMPORT FAILED:", e)
EOF

>> Python 3.8 ortamı aktive ediliyor...
>> Python 3.8 geliştirme dosyaları yükleniyor...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libpython3.8 libpython3.8-dev
The following NEW packages will be installed:
  libpython3.8 libpython3.8-dev python3.8-dev
0 upgraded, 3 newly installed, 0 to remove and 54 not upgraded.
Need to get 6,687 kB of archives.
After this operation, 25.0 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.8 amd64 3.8.20-1+jammy1 [1,798 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.8-dev amd64 3.8.20-1+jammy1 [4,389 kB]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.8-dev amd64 3.8.20-1+jammy1 [500 kB]
Fetched 6,687 kB in 14s (487 kB/s)
Selecting previously unselected package libpython3.8:amd64.
(Reading databa

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openxlab 0.1.3 requires setuptools~=60.2.0, but you have setuptools 75.3.2 which is incompatible.
  Running command git version
  git version 2.34.1
  Running command git clone --filter=blob:none https://github.com/jwwangchn/cocoapi-aitod.git /tmp/pip-req-build-ewwaaqdq
  Cloning into '/tmp/pip-req-build-ewwaaqdq'...
  Running command git rev-parse HEAD
  44a230ae5197cb89bf9e5e62f313cac3ad30c7af
  Running command git rev-parse HEAD
  44a230ae5197cb89bf9e5e62f313cac3ad30c7af
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-d105o67j/aitodpycocotools.egg-info
  writing /tmp/pip-pip-egg-info-d105o67j/aitodpycocotools.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-d105o67j/aitodpycocotools.egg-info/dependency_links.txt
  writing requirements to /tmp/p

In [4]:
%%bash
set -e
source /content/py38/bin/activate

python - << 'EOF'
import sys
print("Python:", sys.version)
try:
    import aitodpycocotools
    print("aitodpycocotools OK:", aitodpycocotools.__file__)
except Exception as e:
    print("aitodpycocotools IMPORT FAILED:", e)
EOF


Python: 3.8.20 (default, Sep  7 2024, 18:35:08) 
[GCC 11.4.0]
aitodpycocotools OK: /content/py38/lib/python3.8/site-packages/aitodpycocotools/__init__.py


In [ ]:
# ============================================================================
# CELL 4: Verify Dataset Location
# ============================================================================
import os
import sys

# Ensure we're using Python 3.8 venv packages if available
if os.path.exists(VENV_PATH):
    venv_site_packages = f'{VENV_PATH}/lib/python3.8/site-packages'
    if venv_site_packages not in sys.path:
        sys.path.insert(0, venv_site_packages)

# Verify dataset is available at the expected location
if os.path.exists(DATASET_EXTRACTED_PATH):
    print(f"✓ Dataset found at: {DATASET_EXTRACTED_PATH}")
    if os.path.exists(f'{DATASET_EXTRACTED_PATH}/annotations'):
        print("✓ Dataset annotations found")
    if os.path.exists(f'{DATASET_EXTRACTED_PATH}/trainval'):
        print("✓ Train/val images found")
    if os.path.exists(f'{DATASET_EXTRACTED_PATH}/test'):
        print("✓ Test images found")
    print(f"\nDataset is ready. Config files should point to: {DATASET_EXTRACTED_PATH}")
else:
    print(f"Note: Dataset not found at {DATASET_EXTRACTED_PATH}")
    print("Make sure the dataset is already extracted in the mmdet-nwdrka/data folder")


✓ Dataset found at: /content/drive/MyDrive/mmdet_nwdrka/mmdet-nwdrka/data/aitodv2
✓ Dataset annotations found
✓ Train/val images found

Dataset is ready. Config files should point to: /content/drive/MyDrive/mmdet_nwdrka/mmdet-nwdrka/data/aitodv2


In [ ]:
%%bash
set -e
source /content/py38/bin/activate
python -m pip install "yapf==0.30.0"

In [ ]:
# ============================================================================
# CELL 5: Train
# ============================================================================
import os
import sys

# Ensure we're using Python 3.8 venv packages if available
if os.path.exists(VENV_PATH):
    venv_site_packages = f'{VENV_PATH}/lib/python3.8/site-packages'
    if venv_site_packages not in sys.path:
        sys.path.insert(0, venv_site_packages)

# Config file (use as-is, no modifications)
CONFIG_FULL = f'{REPO_PATH}/configs_nwdrka/{CONFIG_FILE}'
if not os.path.exists(CONFIG_FULL):
    print(f"ERROR: Config file not found at {CONFIG_FULL}")
    print("Available configs:")
    !find {REPO_PATH}/configs_nwdrka -name "*.py" | head -20
else:
    # Set work directory
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    WORK_DIR_OUTPUT = f'{OUTPUT_PATH}/work_dirs'
    os.makedirs(WORK_DIR_OUTPUT, exist_ok=True)
    TRAIN_WORK_DIR = f'{WORK_DIR_OUTPUT}/train_{os.path.basename(CONFIG_FILE).replace(".py", "")}'

    print(f"Starting training...")
    print(f"Config: {CONFIG_FULL}")
    print(f"Work dir: {TRAIN_WORK_DIR}")
    print(f"Using Python: {VENV_PYTHON}")




Starting training...
Config: /content/drive/MyDrive/mmdet_nwdrka/mmdet-nwdrka/configs_nwdrka/nwd_rka/aitod_super_faster_r50_iou_1x.py
Work dir: /content/drive/MyDrive/outputs/work_dirs/train_aitod_super_faster_r50_iou_1x
Using Python: /content/py38/bin/python


In [ ]:
# Train using config directly (no modifications) with Python 3.8 venv
!{VENV_ACTIVATE} && cd {REPO_PATH} && python tools/train.py configs_nwdrka/{CONFIG_FILE} --work-dir {TRAIN_WORK_DIR}

/content/py38/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/content/drive/MyDrive/mmdet_nwdrka/mmdet-nwdrka/tools/../mmdet/utils/setup_env.py:38: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
/content/drive/MyDrive/mmdet_nwdrka/mmdet-nwdrka/tools/../mmdet/utils/setup_env.py:48: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the vari

In [ ]:
 # Train using config directly (no modifications)
!!source /content/py38/bin/activate && python tools/train.py configs_nwdrka/nwd_rka/aitod_super_faster_r50_nwdrka_1x.py --work-dir /content/drive/MyDrive/outputs/work_dirs/train_aitod_super_faster_r50_nwdrka_1x --gpu-ids 0

["python: can't open file 'tools/train.py': [Errno 2] No such file or directory"]

In [ ]:
# ============================================================================
# CELL 6: Test/Evaluate
# ============================================================================
import os
import glob

# Find latest checkpoint
checkpoint_files = glob.glob(f'{TRAIN_WORK_DIR}/*.pth')
if checkpoint_files:
    checkpoint_files.sort(key=os.path.getmtime, reverse=True)
    LATEST_CHECKPOINT = checkpoint_files[0]
    print(f"Using checkpoint: {LATEST_CHECKPOINT}")

    # Test
    TEST_WORK_DIR = f'{WORK_DIR_OUTPUT}/test_{os.path.basename(CONFIG_FILE).replace(".py", "")}'
    os.makedirs(TEST_WORK_DIR, exist_ok=True)

    # Use original config file (no modifications) with Python 3.8 venv
    !{VENV_ACTIVATE} && cd {REPO_PATH} && python tools/test.py configs_nwdrka/{CONFIG_FILE} \
        {LATEST_CHECKPOINT} \
        --work-dir {TEST_WORK_DIR} \
        --eval bbox \
        --gpu-id 0 \
        --out {TEST_WORK_DIR}/results.pkl

    print(f"\nTesting completed! Results in: {TEST_WORK_DIR}")
else:
    print("No checkpoint found for testing")


No checkpoint found for testing


In [ ]:
checkpoint_files

[]

In [ ]:
# ============================================================================
# CELL 7: Verify Outputs in Drive
# ============================================================================
import os
import glob

print("=" * 80)
print("OUTPUT SUMMARY")
print("=" * 80)
print(f"\nOutputs saved to Google Drive:")
print(f"  Training outputs: {TRAIN_WORK_DIR}")

if os.path.exists(f'{WORK_DIR_OUTPUT}/test_{os.path.basename(CONFIG_FILE).replace(".py", "")}'):
    print(f"  Test outputs: {WORK_DIR_OUTPUT}/test_{os.path.basename(CONFIG_FILE).replace(".py", "")}")

# List checkpoints
checkpoints = glob.glob(f'{TRAIN_WORK_DIR}/*.pth')
if checkpoints:
    print(f"\nFound {len(checkpoints)} checkpoint(s):")
    for ckpt in sorted(checkpoints, key=os.path.getmtime, reverse=True)[:5]:  # Show first 5
        size_mb = os.path.getsize(ckpt) / (1024 * 1024)
        mtime = os.path.getmtime(ckpt)
        from datetime import datetime
        mtime_str = datetime.fromtimestamp(mtime).strftime('%Y-%m-%d %H:%M:%S')
        print(f"  {os.path.basename(ckpt)} ({size_mb:.1f} MB, {mtime_str})")

print("\n" + "=" * 80)
print("All outputs are saved in your Google Drive!")
print("=" * 80)


## Additional Notes:

### To Resume Training:
Modify CELL 5 to add `--resume-from {checkpoint_path}` to the training command.

### To Use a Different Checkpoint for Testing:
Modify CELL 6 to specify a different checkpoint path instead of using `LATEST_CHECKPOINT`.

### To Adjust Training Parameters:
You can modify the config file or add command-line arguments in CELL 5:
- `--resume-from`: Resume from a checkpoint
- `--auto-resume`: Auto-resume from latest checkpoint
- `--no-validate`: Skip validation during training

### Memory Issues:
If you run out of memory, reduce batch size in the config file by modifying `samples_per_gpu` in the dataset configuration.


## TESTİNG WİTH VİSUALİZATİON İMAGE

In [35]:
script_path = "/content/drive/MyDrive/mmdet_nwdrka/mmdet-nwdrka/infer_batch.py"

script_code = r'''
import os
import sys
from pathlib import Path
import glob

# -------------------- USER PATHS --------------------
MMDET_REPO_ROOT = r"/content/drive/MyDrive/mmdet_nwdrka/mmdet-nwdrka"
CFG_PATH        = r"/content/drive/MyDrive/outputs/work_dirs/train_aitod_super_faster_r50_csam_nwdrka_1x/aitod_super_faster_r50_csam_nwdrka_1x.py"
CKPT_PATH       = r"/content/drive/MyDrive/outputs/work_dirs/train_aitod_super_faster_r50_csam_nwdrka_1x/epoch_10.pth"

# Tek resim yerine KLASÖR
IMG_DIR         = r"/content/drive/MyDrive/mmdet_nwdrka/mmdet-nwdrka/data/aitod_super/stitched_val"  # <--- klasör yolu
OUT_DIR         = r"/content/drive/MyDrive/mmdet_nwdrka/mmdet-nwdrka/data/aitod_super/stitched_val_pred_nwdrka_normal"

SCORE_THR       = 0.80
DEVICE          = "cuda:0"   # ya da "cuda" / "cpu"

# -------------------- ENV SETUP --------------------
MMDET_REPO_ROOT = Path(MMDET_REPO_ROOT).expanduser().resolve()
assert MMDET_REPO_ROOT.exists(), f"Repo root does not exist: {MMDET_REPO_ROOT}"

if str(MMDET_REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(MMDET_REPO_ROOT))

os.chdir(MMDET_REPO_ROOT)

CFG_PATH  = str(Path(CFG_PATH).expanduser().resolve())
CKPT_PATH = str(Path(CKPT_PATH).expanduser().resolve())
IMG_DIR   = str(Path(IMG_DIR).expanduser().resolve())
OUT_DIR   = str(Path(OUT_DIR).expanduser().resolve())

assert os.path.isfile(CFG_PATH),  f"Config not found: {CFG_PATH}"
assert os.path.isfile(CKPT_PATH), f"Checkpoint not found: {CKPT_PATH}"
assert os.path.isdir(IMG_DIR),    f"Image folder not found: {IMG_DIR}"

os.makedirs(OUT_DIR, exist_ok=True)

# -------------------- IMPORTS (MMDet 2.x) --------------------
from mmdet.apis import init_detector, inference_detector
import mmcv

# -------------------- MODEL INIT --------------------
model = init_detector(CFG_PATH, CKPT_PATH, device=DEVICE)

# -------------------- BATCH INFERENCE --------------------
# Desteklediğin uzantıları ekleyebilirsin
img_extensions = ["*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tif", "*.tiff"]

img_paths = []
for ext in img_extensions:
    img_paths.extend(glob.glob(os.path.join(IMG_DIR, ext)))

img_paths = sorted(img_paths)
print(f"Found {len(img_paths)} images under {IMG_DIR}")

for img_path in img_paths:
    img_name = os.path.basename(img_path)
    out_file = os.path.join(OUT_DIR, img_name)

    # Inference
    result = inference_detector(model, img_path)

    # MMDet 2.26: show_result imzası:
    # show_result(img, result, score_thr=0.3, show=False, win_name='', wait_time=0, out_file=None)
    model.show_result(
        img_path,
        result,
        score_thr=SCORE_THR,
        show=False,          # Colab'de pop-up istemiyoruz
        out_file=out_file,   # çıktı görüntü yolu
    )

    print(f"[OK] Saved: {out_file}")

print("All done.")
'''

with open(script_path, "w") as f:
    f.write(script_code)

print("Saved script to:", script_path)


Saved script to: /content/drive/MyDrive/mmdet_nwdrka/mmdet-nwdrka/infer_batch.py


In [36]:
!source "/content/py38/bin/activate" && pip install matplotlib_inline IPython

In [37]:
%%bash
VENV_PATH="/content/py38"
REPO_PATH="/content/drive/MyDrive/mmdet_nwdrka/mmdet-nwdrka"
SCRIPT_PATH="infer_batch.py"

source "${VENV_PATH}/bin/activate"

python -V
which python

cd "${REPO_PATH}"
python "${SCRIPT_PATH}"


Process is terminated.
